In [1]:
import torch
import numpy as np
import pandas as pd
import os
import time
from glob import glob
from itertools import cycle

In [2]:
dirs = '../M5_Dataset'

In [3]:
INPUT_DIR = dirs

calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
calendar.name = 'calendar'

prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
prices.name = 'prices'

sales = pd.read_csv(f'{INPUT_DIR}/sales_train_evaluation.csv')
sales.name = 'sales'

In [4]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

In [5]:
df = sales.groupby(['store_id', 'dept_id']).sum()

index_list = df.index

df.reset_index(inplace=True)


idx_list = []
state_list = []
cat_list = []

for i in range(len(index_list)):
    idx = index_list[i][0] + '_' + index_list[i][1]
    idx_list.append(idx)
    
    state = df.iloc[i]['store_id'].split('_')
    state_list.append(state[0])
    
    cat = df.iloc[i]['dept_id'].split('_')
    cat_list.append(cat[0])
    
df['Node'] = idx_list
df['Node'] = df['Node'].astype('category')

# df.set_index('index', inplace=True)

In [6]:
df['state_id'] = state_list
df['state_id'] = df['state_id'].astype('category')

df['cat_id'] = cat_list
df['cat_id'] = df['cat_id'].astype('category')

In [7]:
df

,store_id,dept_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,Node,state_id,cat_id
0,CA_1,FOODS_1,297,284,214,175,182,191,224,263,...,277.0,302.0,397.0,330.0,310.0,427,334.0,CA_1_FOODS_1,CA,FOODS
1,CA_1,FOODS_2,674,655,396,476,354,486,554,747,...,555.0,476.0,427.0,411.0,434.0,627,719.0,CA_1_FOODS_2,CA,FOODS
2,CA_1,FOODS_3,2268,2198,1398,1607,1496,1730,1915,2952,...,2366.0,2016.0,1992.0,2113.0,2249.0,2729,3274.0,CA_1_FOODS_3,CA,FOODS
3,CA_1,HOBBIES_1,528,489,409,383,263,453,339,750,...,389.0,347.0,353.0,354.0,494.0,656,588.0,CA_1_HOBBIES_1,CA,HOBBIES
4,CA_1,HOBBIES_2,28,9,6,9,5,14,7,19,...,28.0,46.0,28.0,52.0,45.0,63,58.0,CA_1_HOBBIES_2,CA,HOBBIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,WI_3,FOODS_3,2293,2383,1841,1965,1427,2760,2727,3786,...,1952.0,1602.0,1594.0,1627.0,2011.0,2360,2632.0,WI_3_FOODS_3,WI,FOODS
66,WI_3,HOBBIES_1,256,342,228,183,70,285,357,399,...,187.0,174.0,287.0,220.0,242.0,273,305.0,WI_3_HOBBIES_1,WI,HOBBIES
67,WI_3,HOBBIES_2,22,14,20,11,4,8,3,13,...,11.0,45.0,45.0,46.0,27.0,31,37.0,WI_3_HOBBIES_2,WI,HOBBIES
68,WI_3,HOUSEHOLD_1,584,541,420,327,151,392,492,647,...,626.0,597.0,486.0,595.0,764.0,723,723.0,WI_3_HOUSEHOLD_1,WI,HOUSEHOLD


In [8]:
df_melt = pd.melt(df, id_vars=['store_id', 'dept_id', 'state_id', 'cat_id','Node'], var_name='d', value_name='sold').dropna()

In [9]:
df_melt

,store_id,dept_id,state_id,cat_id,Node,d,sold
0,CA_1,FOODS_1,CA,FOODS,CA_1_FOODS_1,d_1,297.0
1,CA_1,FOODS_2,CA,FOODS,CA_1_FOODS_2,d_1,674.0
2,CA_1,FOODS_3,CA,FOODS,CA_1_FOODS_3,d_1,2268.0
3,CA_1,HOBBIES_1,CA,HOBBIES,CA_1_HOBBIES_1,d_1,528.0
4,CA_1,HOBBIES_2,CA,HOBBIES,CA_1_HOBBIES_2,d_1,28.0
...,...,...,...,...,...,...,...
135865,WI_3,FOODS_3,WI,FOODS,WI_3_FOODS_3,d_1941,2632.0
135866,WI_3,HOBBIES_1,WI,HOBBIES,WI_3_HOBBIES_1,d_1941,305.0
135867,WI_3,HOBBIES_2,WI,HOBBIES,WI_3_HOBBIES_2,d_1941,37.0
135868,WI_3,HOUSEHOLD_1,WI,HOUSEHOLD,WI_3_HOUSEHOLD_1,d_1941,723.0


In [10]:
df_concat_calendar = pd.merge(df_melt, calendar, on='d', how='left')

In [11]:
df_concat_calendar

,store_id,dept_id,state_id,cat_id,Node,d,sold,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,CA_1,FOODS_1,CA,FOODS,CA_1_FOODS_1,d_1,297.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,CA_1,FOODS_2,CA,FOODS,CA_1_FOODS_2,d_1,674.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,CA_1,FOODS_3,CA,FOODS,CA_1_FOODS_3,d_1,2268.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,CA_1,HOBBIES_1,CA,HOBBIES,CA_1_HOBBIES_1,d_1,528.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,CA_1,HOBBIES_2,CA,HOBBIES,CA_1_HOBBIES_2,d_1,28.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135865,WI_3,FOODS_3,WI,FOODS,WI_3_FOODS_3,d_1941,2632.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0
135866,WI_3,HOBBIES_1,WI,HOBBIES,WI_3_HOBBIES_1,d_1941,305.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0
135867,WI_3,HOBBIES_2,WI,HOBBIES,WI_3_HOBBIES_2,d_1941,37.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0
135868,WI_3,HOUSEHOLD_1,WI,HOUSEHOLD,WI_3_HOUSEHOLD_1,d_1941,723.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0


In [12]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [13]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

In [14]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [15]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [16]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,...,5,2016,NaN,NaN,NaN,NaN,0,0,0,2.980469
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,5,2016,NaN,NaN,NaN,NaN,0,0,0,2.480469
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,...,5,2016,NaN,NaN,NaN,NaN,0,0,0,3.980469
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,5,2016,NaN,NaN,NaN,NaN,0,0,0,1.280273


In [183]:
temp = df_concat_calendar

In [184]:
d_dept_id = dict(zip(temp.dept_id.cat.codes, temp.dept_id))
d_cat_id = dict(zip(temp.cat_id.cat.codes, temp.cat_id))
d_store_id = dict(zip(temp.store_id.cat.codes, temp.store_id))
d_state_id = dict(zip(temp.state_id.cat.codes, temp.state_id))
d_node = dict(zip(temp.Node.cat.codes, temp.Node))

In [185]:
temp.d = temp['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [186]:
cols = temp.dtypes.index.tolist()
cols

['store_id',
 'dept_id',
 'state_id',
 'cat_id',
 'Node',
 'd',
 'sold',
 'date',
 'wm_yr_wk',
 'weekday',
 'wday',
 'month',
 'year',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI']

In [187]:
types = temp.dtypes.values.tolist()
types

[CategoricalDtype(categories=['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3',
                   'WI_1', 'WI_2', 'WI_3'],
 , ordered=False),
 CategoricalDtype(categories=['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2',
                   'HOUSEHOLD_1', 'HOUSEHOLD_2'],
 , ordered=False),
 CategoricalDtype(categories=['CA', 'TX', 'WI'], ordered=False),
 CategoricalDtype(categories=['FOODS', 'HOBBIES', 'HOUSEHOLD'], ordered=False),
 CategoricalDtype(categories=['CA_1_FOODS_1', 'CA_1_FOODS_2', 'CA_1_FOODS_3',
                   'CA_1_HOBBIES_1', 'CA_1_HOBBIES_2', 'CA_1_HOUSEHOLD_1',
                   'CA_1_HOUSEHOLD_2', 'CA_2_FOODS_1', 'CA_2_FOODS_2',
                   'CA_2_FOODS_3', 'CA_2_HOBBIES_1', 'CA_2_HOBBIES_2',
                   'CA_2_HOUSEHOLD_1', 'CA_2_HOUSEHOLD_2', 'CA_3_FOODS_1',
                   'CA_3_FOODS_2', 'CA_3_FOODS_3', 'CA_3_HOBBIES_1',
                   'CA_3_HOBBIES_2', 'CA_3_HOUSEHOLD_1', 'CA_3_HOUSEHOLD_2',
                   'CA_4_FOODS_1

In [188]:
for i,type in enumerate(types):
    if type.name == 'category':
        temp[cols[i]] = temp[cols[i]].cat.codes

In [190]:
temp.drop('date',axis=1,inplace=True)
temp.drop('wm_yr_wk',axis=1,inplace=True)

In [191]:
temp

,store_id,dept_id,state_id,cat_id,Node,d,sold,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,0,0,0,0,0,1,297.0,2,1,1,2011,-1,-1,-1,-1,0,0,0
1,0,1,0,0,1,1,674.0,2,1,1,2011,-1,-1,-1,-1,0,0,0
2,0,2,0,0,2,1,2268.0,2,1,1,2011,-1,-1,-1,-1,0,0,0
3,0,3,0,1,3,1,528.0,2,1,1,2011,-1,-1,-1,-1,0,0,0
4,0,4,0,1,4,1,28.0,2,1,1,2011,-1,-1,-1,-1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135865,9,2,2,0,65,1941,2632.0,3,2,5,2016,-1,-1,-1,-1,0,0,0
135866,9,3,2,1,66,1941,305.0,3,2,5,2016,-1,-1,-1,-1,0,0,0
135867,9,4,2,1,67,1941,37.0,3,2,5,2016,-1,-1,-1,-1,0,0,0
135868,9,5,2,2,68,1941,723.0,3,2,5,2016,-1,-1,-1,-1,0,0,0


In [195]:
temp.keys()

Index(['store_id', 'dept_id', 'state_id', 'cat_id', 'Node', 'd', 'sold',
       'weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object')

In [196]:
dataset = temp[['Node','d', 'store_id', 'dept_id', 'state_id', 'cat_id',
       'weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sold']]

In [197]:
dataset

,Node,d,store_id,dept_id,state_id,cat_id,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sold
0,0,1,0,0,0,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,297.0
1,1,1,0,1,0,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,674.0
2,2,1,0,2,0,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,2268.0
3,3,1,0,3,0,1,2,1,1,2011,-1,-1,-1,-1,0,0,0,528.0
4,4,1,0,4,0,1,2,1,1,2011,-1,-1,-1,-1,0,0,0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135865,65,1941,9,2,2,0,3,2,5,2016,-1,-1,-1,-1,0,0,0,2632.0
135866,66,1941,9,3,2,1,3,2,5,2016,-1,-1,-1,-1,0,0,0,305.0
135867,67,1941,9,4,2,1,3,2,5,2016,-1,-1,-1,-1,0,0,0,37.0
135868,68,1941,9,5,2,2,3,2,5,2016,-1,-1,-1,-1,0,0,0,723.0


In [198]:
# sales.to_pickle('../M5_Dataset/sales.pkl')
# prices.to_pickle('../M5_Dataset/prices.pkl')
# calendar.to_pickle('../M5_Dataset/calendar.pkl')

temp.to_pickle('../M5_Dataset/dataset.pkl')

## Prices

In [70]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [82]:
prices['dept_id'] = prices['item_id'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1]).astype('category')

In [83]:
prices

,store_id,item_id,wm_yr_wk,sell_price,dept_id
0,CA_1,HOBBIES_1_001,11325,9.578125,HOBBIES_1
1,CA_1,HOBBIES_1_001,11326,9.578125,HOBBIES_1
2,CA_1,HOBBIES_1_001,11327,8.257812,HOBBIES_1
3,CA_1,HOBBIES_1_001,11328,8.257812,HOBBIES_1
4,CA_1,HOBBIES_1_001,11329,8.257812,HOBBIES_1
...,...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000,FOODS_3
6841117,WI_3,FOODS_3_827,11618,1.000000,FOODS_3
6841118,WI_3,FOODS_3_827,11619,1.000000,FOODS_3
6841119,WI_3,FOODS_3_827,11620,1.000000,FOODS_3


In [87]:
prices.drop(['item_id'], axis=1, inplace=True)

In [88]:
prices

,store_id,wm_yr_wk,sell_price,dept_id
0,CA_1,11325,9.578125,HOBBIES_1
1,CA_1,11326,9.578125,HOBBIES_1
2,CA_1,11327,8.257812,HOBBIES_1
3,CA_1,11328,8.257812,HOBBIES_1
4,CA_1,11329,8.257812,HOBBIES_1
...,...,...,...,...
6841116,WI_3,11617,1.000000,FOODS_3
6841117,WI_3,11618,1.000000,FOODS_3
6841118,WI_3,11619,1.000000,FOODS_3
6841119,WI_3,11620,1.000000,FOODS_3


In [98]:
price_groupby = prices.groupby(['store_id', 'wm_yr_wk', 'dept_id']).mean()

In [ ]:
price_groupby['store_id']

In [99]:
1941*70

135870

In [100]:
price_groupby

sell_price
store_id wm_yr_wk dept_id                
CA_1     11101    FOODS_1        2.988281
                  FOODS_2        3.808594
                  FOODS_3        2.830078
                  HOBBIES_1      4.773438
                  HOBBIES_2      2.982422
...                                   ...
WI_3     11621    FOODS_3        2.912109
                  HOBBIES_1      6.609375
                  HOBBIES_2      2.578125
                  HOUSEHOLD_1    5.183594
                  HOUSEHOLD_2    5.773438

[19740 rows x 1 columns]

# M5 Dataset --> Pytorch Forecasting TumeSeries Dataset

In [123]:
dirs = '../M5_Dataset'

sales = pd.read_pickle(dirs+'/sales.pkl')
calendar = pd.read_pickle(dirs+'/calendar.pkl')

In [124]:
calendar[event_day] = calendar[event_day].astype(str)

calendar['event_name_1'] = calendar['event_name_1'].apply(lambda x: '-' if x == 'nan' else x).astype("category")
calendar['event_name_2'] = calendar['event_name_2'].apply(lambda x: '-' if x == 'nan' else x).astype("category")
calendar['event_type_1'] = calendar['event_type_1'].apply(lambda x: '-' if x == 'nan' else x).astype("category")
calendar['event_type_2'] = calendar['event_type_2'].apply(lambda x: '-' if x == 'nan' else x).astype("category")

In [125]:
calendar['month'] = calendar['month'].astype('category')
calendar['wday'] = calendar['wday'].astype('category')
calendar.drop(['wm_yr_wk', 'year'], axis=1, inplace=True)

In [126]:
snap_day = ['snap_CA', 'snap_TX', 'snap_WI']

calendar[snap_day] = calendar[snap_day].astype("category")

In [127]:
calendar

,date,weekday,wday,month,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,Saturday,1,1,d_1,-,-,-,-,0,0,0
1,2011-01-30,Sunday,2,1,d_2,-,-,-,-,0,0,0
2,2011-01-31,Monday,3,1,d_3,-,-,-,-,0,0,0
3,2011-02-01,Tuesday,4,2,d_4,-,-,-,-,1,1,0
4,2011-02-02,Wednesday,5,2,d_5,-,-,-,-,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,Wednesday,5,6,d_1965,-,-,-,-,0,1,1
1965,2016-06-16,Thursday,6,6,d_1966,-,-,-,-,0,0,0
1966,2016-06-17,Friday,7,6,d_1967,-,-,-,-,0,0,0
1967,2016-06-18,Saturday,1,6,d_1968,-,-,-,-,0,0,0


In [128]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   weekday       1969 non-null   category      
 2   wday          1969 non-null   category      
 3   month         1969 non-null   category      
 4   d             1969 non-null   category      
 5   event_name_1  1969 non-null   category      
 6   event_type_1  1969 non-null   category      
 7   event_name_2  1969 non-null   category      
 8   event_type_2  1969 non-null   category      
 9   snap_CA       1969 non-null   category      
 10  snap_TX       1969 non-null   category      
 11  snap_WI       1969 non-null   category      
dtypes: category(11), datetime64[ns](1)
memory usage: 121.8 KB


In [129]:
df = sales.groupby(['store_id', 'dept_id']).sum()

index_list = df.index

df.reset_index(inplace=True)


idx_list = []
state_list = []
cat_list = []

for i in range(len(index_list)):
    idx = index_list[i][0] + '_' + index_list[i][1]
    idx_list.append(idx)
    
    state = df.iloc[i]['store_id'].split('_')
    state_list.append(state[0])
    
    cat = df.iloc[i]['dept_id'].split('_')
    cat_list.append(cat[0])
    
df['Node'] = idx_list
df['Node'] = df['Node'].astype('category')

df['state_id'] = state_list
df['state_id'] = df['state_id'].astype('category')

df['cat_id'] = cat_list
df['cat_id'] = df['cat_id'].astype('category')

df_melt = pd.melt(df, id_vars=['store_id', 'dept_id', 'state_id', 'cat_id','Node'], var_name='d', value_name='sold').dropna()

df_concat_calendar = pd.merge(df_melt, calendar, on='d', how='left')


In [130]:
df_concat_calendar.d = df_concat_calendar['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [131]:
df_concat_calendar

,store_id,dept_id,state_id,cat_id,Node,d,sold,date,weekday,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,CA_1,FOODS_1,CA,FOODS,CA_1_FOODS_1,1,297.0,2011-01-29,Saturday,1,1,-,-,-,-,0,0,0
1,CA_1,FOODS_2,CA,FOODS,CA_1_FOODS_2,1,674.0,2011-01-29,Saturday,1,1,-,-,-,-,0,0,0
2,CA_1,FOODS_3,CA,FOODS,CA_1_FOODS_3,1,2268.0,2011-01-29,Saturday,1,1,-,-,-,-,0,0,0
3,CA_1,HOBBIES_1,CA,HOBBIES,CA_1_HOBBIES_1,1,528.0,2011-01-29,Saturday,1,1,-,-,-,-,0,0,0
4,CA_1,HOBBIES_2,CA,HOBBIES,CA_1_HOBBIES_2,1,28.0,2011-01-29,Saturday,1,1,-,-,-,-,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135865,WI_3,FOODS_3,WI,FOODS,WI_3_FOODS_3,1941,2632.0,2016-05-22,Sunday,2,5,-,-,-,-,0,0,0
135866,WI_3,HOBBIES_1,WI,HOBBIES,WI_3_HOBBIES_1,1941,305.0,2016-05-22,Sunday,2,5,-,-,-,-,0,0,0
135867,WI_3,HOBBIES_2,WI,HOBBIES,WI_3_HOBBIES_2,1941,37.0,2016-05-22,Sunday,2,5,-,-,-,-,0,0,0
135868,WI_3,HOUSEHOLD_1,WI,HOUSEHOLD,WI_3_HOUSEHOLD_1,1941,723.0,2016-05-22,Sunday,2,5,-,-,-,-,0,0,0


In [132]:
df_concat_calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135870 entries, 0 to 135869
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   store_id      135870 non-null  category      
 1   dept_id       135870 non-null  category      
 2   state_id      135870 non-null  category      
 3   cat_id        135870 non-null  category      
 4   Node          135870 non-null  category      
 5   d             135870 non-null  int16         
 6   sold          135870 non-null  float64       
 7   date          135870 non-null  datetime64[ns]
 8   weekday       135870 non-null  category      
 9   wday          135870 non-null  category      
 10  month         135870 non-null  category      
 11  event_name_1  135870 non-null  category      
 12  event_type_1  135870 non-null  category      
 13  event_name_2  135870 non-null  category      
 14  event_type_2  135870 non-null  category      
 15  snap_CA       135

In [133]:
df_concat_calendar.to_pickle(dirs+'/tft_dataset.pkl')

## Level 12 Data preprocessing

In [17]:
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            category      
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             object        
 7   sold          int16         
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       category      
 11  wday          int8          
 12  month         int8          
 13  year          int16         
 14  event_name_1  category      
 15  event_type_1  category      
 16  event_name_2  category      
 17  event_type_2  category      
 18  snap_CA       int8          
 19  snap_TX       int8          
 20  snap_WI       int8          
 21  sell_price    float16       
dtypes: category(11), datetime64[ns](1), float16(1), int16(3), int8(5

In [20]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        
#3
df.drop('date',axis=1,inplace=True)

In [21]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,14370,1437,3,1,0,0,1,0,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,NaN
1,14380,1438,3,1,0,0,1,0,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,NaN
2,14390,1439,3,1,0,0,1,0,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,NaN
3,14400,1440,3,1,0,0,1,0,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,NaN
4,14410,1441,3,1,0,0,1,0,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,14329,1432,2,0,9,2,1941,1,11617,3,...,5,2016,-1,-1,-1,-1,0,0,0,2.980469
59181086,14339,1433,2,0,9,2,1941,0,11617,3,...,5,2016,-1,-1,-1,-1,0,0,0,2.480469
59181087,14349,1434,2,0,9,2,1941,2,11617,3,...,5,2016,-1,-1,-1,-1,0,0,0,3.980469
59181088,14359,1435,2,0,9,2,1941,0,11617,3,...,5,2016,-1,-1,-1,-1,0,0,0,1.280273


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 21 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int16  
 1   item_id       int16  
 2   dept_id       int8   
 3   cat_id        int8   
 4   store_id      int8   
 5   state_id      int8   
 6   d             int16  
 7   sold          int16  
 8   wm_yr_wk      int16  
 9   weekday       int8   
 10  wday          int8   
 11  month         int8   
 12  year          int16  
 13  event_name_1  int8   
 14  event_type_1  int8   
 15  event_name_2  int8   
 16  event_type_2  int8   
 17  snap_CA       int8   
 18  snap_TX       int8   
 19  snap_WI       int8   
 20  sell_price    float16
dtypes: float16(1), int16(6), int8(14)
memory usage: 2.0 GB


In [23]:
df.drop('wm_yr_wk',axis=1,inplace=True)

In [24]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,14370,1437,3,1,0,0,1,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
1,14380,1438,3,1,0,0,1,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
2,14390,1439,3,1,0,0,1,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
3,14400,1440,3,1,0,0,1,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
4,14410,1441,3,1,0,0,1,0,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,14329,1432,2,0,9,2,1941,1,3,2,5,2016,-1,-1,-1,-1,0,0,0,2.980469
59181086,14339,1433,2,0,9,2,1941,0,3,2,5,2016,-1,-1,-1,-1,0,0,0,2.480469
59181087,14349,1434,2,0,9,2,1941,2,3,2,5,2016,-1,-1,-1,-1,0,0,0,3.980469
59181088,14359,1435,2,0,9,2,1941,0,3,2,5,2016,-1,-1,-1,-1,0,0,0,1.280273


In [27]:
df.fillna(0, inplace =True)

In [28]:
df.to_pickle('../M5_Dataset/level_12.pkl')